### Topic Modeling Evaluation to Train LDA with OCTIS

Welcome! This is a notebook that allows you to train a topic model using OCTIS (Optimizing and Comparing Topic Models Is Simple). 

A topic model allows you to discover the latent topics in your documents in a completely unsupervised way. Just use your documents and get topics out! It's very easy with OCTIS :)

Let's start! First, we need to install OCTIS. (We are going to use the library version of OCTIS, but you can also use it through its dashboard. See https://github.com/mind-Lab/octis for more details.)

In [ ]:
!pip install octis

In [ ]:
!pip uninstall numpy
!pip install numpy

Let's import what we need. 

In [1]:
from octis.models.LDA import LDA
from octis.dataset.dataset import Dataset
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence

/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


We need some data to run a topic model. OCTIS already provides 4 already-preprocessed datasets. Let's use one of them.

In [2]:
# Define dataset
dataset = Dataset()
dataset.fetch_dataset("20NewsGroup")

And now we need a model. We are going to use LDA because it is the most well-known, but OCTIS integrates other 8 topic model (including neural topic models!). 

We are going to set the number of topics to 20 and the hyperparameter alpha to 0.1. If you have no idea how to set your hyperparameters, you should definitely use OCTIS's optimization module. See this other tutorial for the optimization of hyperparameters: (link)

In [5]:
# Create Model
model = LDA(num_topics=20, alpha=0.1,passes=10)

Now we're ready to train it. See that the output of a topic model comes as a dictionary composed of 4 elements:


*   *topics*: the list of word topics
*   *topic-word-matrix*: the distribution of the words of the vocabulary for each topic (dimensions: |num topics| x |vocabulary|)
*   *topic-document-matrix*: the distribution of the topics for each document of the training set (dimensions: |num topics| x |training documents|)
*   *test-document-topic-matrix*: the distribution of the topics for each document of the testing set (dimensions: |num topics| x |test documents|)



In [6]:
# Train the model using default partitioning choice 
output = model.train_model(dataset)

print(*list(output.keys()), sep="\n") # Print the output identifiers

topic-word-matrix
topics
topic-document-matrix
test-topic-document-matrix


For  examples, these are a sample of 5 topics. Do you think they make sense?

In [7]:
for t in output['topics'][:5]:
  print(" ".join(t))

mail information list send address post message group email internet
religion church true people claim belief word make exist evidence
car water buy engine problem good make drive ground power
price sell good offer sale book include mail buy interested
image color bit format program display file line output convert


To check if the topics are coherent, we can use a topic coherence measure. The most used one is NPMI and it is available in OCTIS. We are going to use the dataset itself to compute it. 

In [17]:
# Initialize metric
npmi = Coherence(texts=dataset.get_corpus(), topk=10, measure='c_npmi')
cv = Coherence(texts=dataset.get_corpus(), topk=10, measure='c_v')


Or we can test if the resulting topics are different from each other. The `TopicDiversity` measure computes the number of unique words in the top-words of the resulting topics. 



In [15]:
# Initialize metric
topic_diversity = TopicDiversity(topk=10)

And with the method `score`, we can get their actual evaluation score. Just use the output of the topic model as input of the method.

In [18]:
# Retrieve metrics score
topic_diversity_score = topic_diversity.score(output)
print("Topic diversity: "+str(topic_diversity_score))

npmi_score = npmi.score(output)
print("Coherence npmi: "+str(npmi_score))

cv_score = cv.score(output)
print("Coherence cv: "+str(cv_score))

Topic diversity: 0.795
Coherence npmi: 0.09931255495878394
Coherence cv: 0.6218479785729554
